# Analisi

In [1]:
import geopandas as gpd

from my_paths import *

In [2]:
PATH_ANALISI_TEST_CLEAN = "../Data/Clean/Analisi/test.geojson"

In [3]:
gdf_ciclabili = gpd.read_file(PATH_CICLABILI_CLEAN).to_crs("EPSG:32632")
gdf_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN).to_crs("EPSG:32632")
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN).to_crs("EPSG:32632")
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN).to_crs("EPSG:32632")
gdf_impianti_sportivi = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).to_crs("EPSG:32632")
gdf_fontane = gpd.read_file(PATH_FONTANE_CLEAN).to_crs("EPSG:32632")
gdf_inquinamento = gpd.read_file(PATH_INQUINAMENTO_INGESTION_CLEAN).to_crs("EPSG:32632")

### Metodo per trovare intersezioni tra geometrie.

Per farlo si crea un "buffer" di tot distanza (esempio 1000m) attorno a tutti gli oggetti geometrici.
Se quando si usa "intersects.()" su di un altro oggetto geometrico quello va a intersecarsi con il buffer allora restituisce True altrimenti False. Si crea una Series di True e False.  
Qui usiamo il "not" (~) per capire quale parco NON si interseca con il buffer di 1km per ogni ciclabile e ci restituisce il geodataframe con i parchi più lontani da quella distanza.

In [4]:
buffer_union = gdf_ciclabili.buffer(1000).union_all()
parchi_non_vicini = gdf_parchi[~gdf_parchi.geometry.intersects(buffer_union)]

Per trovare "una strada" che possa connettere i "parchi_non_vicini" per prima cosa ho deciso di trovare il loro "centro".  
Il problema consiste nell'esistenza di tante righe contenente uno stesso parco (dovuto ai vari poligoni).  
Usiamo "geopandas_center" per ricavare il centro di un parco durante il raggruppamento per nome.

In [14]:
def geopandas_center(polygons):
    gdf = gpd.GeoDataFrame(geometry=list(polygons))
    return gdf.union_all().centroid

In [15]:
parchi_non_vicini = parchi_non_vicini.to_crs("EPSG:4326") # Più comodo per coordinate spaziali
parchi_raggruppati = gpd.GeoDataFrame(parchi_non_vicini.groupby("PARCO")['geometry'].agg(geopandas_center)).reset_index()

Salviamo per test

In [ ]:
parchi_raggruppati.to_file(PATH_ANALISI_TEST_CLEAN, driver="GeoJSON")